# Adaptive Stigmergy Engine Demo

**The claim:** The engine activates coordination only when economically justified.

**Two tasks, same architecture, same weights:**
- **Task A (Easy):** Visible food, no hidden food. Gate should CLOSE (~0).
- **Task B (Hard):** Visible food + hidden food requiring 3 agents to reveal. Gate should OPEN (>0.5).

**Success criteria:**
1. `mean_gate(Task B) >= 3 × mean_gate(Task A)`
2. Task B shows hidden food reveals
3. Both populations survive

**Run all cells. ~15 min total.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os

REPO_DIR = '/content/emergence-lab'
GITHUB_USERNAME = 'imashishkh21'

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/{GITHUB_USERNAME}/emergence-lab.git {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull origin main

os.chdir(REPO_DIR)
!pip install -e ".[dev]" -q

import jax
print(f"JAX devices: {jax.devices()}")

Mounted at /content/drive
Cloning into '/content/emergence-lab'...
remote: Enumerating objects: 1640, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 1640 (delta 168), reused 217 (delta 93), pack-reused 1304 (from 1)
Receiving objects: 100% (1640/1640), 2.06 MiB | 29.74 MiB/s, done.
Resolving deltas: 100% (1065/1065), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Ge

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


JAX devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]


In [2]:
from src.configs import Config

TOTAL_STEPS = 3_000_000
NUM_SEEDS = 3
CHECKPOINT_BASE = '/content/drive/MyDrive/emergence-lab/adaptive_gate_demo_v2'


def build_task_a() -> Config:
    """Task A: Easy — coordination NOT needed. Gate should close."""
    config = Config()

    config.env.grid_size = 14
    config.env.num_agents = 8
    config.env.num_food = 12

    # Full food visibility — agents see everything
    config.env.food_obs_enabled = True
    config.env.food_odor_enabled = False

    # No hidden food — no coordination required
    config.env.hidden_food.enabled = False

    # Evolution
    config.evolution.enabled = True
    config.evolution.max_agents = 32
    config.evolution.starting_energy = 300
    config.evolution.max_energy = 300
    config.evolution.food_energy = 100
    config.evolution.reproduce_threshold = 120
    config.evolution.reproduce_cost = 80
    config.evolution.mutation_std = 0.01

    # Training
    config.train.total_steps = TOTAL_STEPS
    config.train.num_envs = 32
    config.train.num_steps = 128
    config.log.wandb = False
    config.log.save_interval = 0

    # Adaptive gate ON — this is what we're testing
    config.field.adaptive_gate = True
    config.field.evolutionary_gate_only = True
    config.field.gate_bias_mutation_std = 0.5
    config.field.channel_diffusion_rates = (0.15, 1.0, 0.0, 0.0)
    config.field.channel_decay_rates = (0.05, 1.0, 0.0, 0.0)

    # No patch scaling
    config.nest.patch_scaling_enabled = False
    config.nest.food_patch_marking = False

    return config


def build_task_b() -> Config:
    """Task B: Hard — hidden food requires 5 agents to reveal. Gate should open."""
    config = Config()

    config.env.grid_size = 14
    config.env.num_agents = 8
    config.env.num_food = 6  # sparse regular food

    # Full food visibility
    config.env.food_obs_enabled = True
    config.env.food_odor_enabled = False

    # Hidden food ON — coordination REQUIRED
    config.env.hidden_food.enabled = True
    config.env.hidden_food.num_hidden = 3
    config.env.hidden_food.required_agents = 5
    config.env.hidden_food.reveal_distance = 1
    config.env.hidden_food.reveal_duration = 10
    config.env.hidden_food.hidden_food_value_multiplier = 5.0

    # Evolution — more energy for coordination task
    config.evolution.enabled = True
    config.evolution.max_agents = 32
    config.evolution.starting_energy = 400
    config.evolution.max_energy = 400
    config.evolution.food_energy = 100
    config.evolution.reproduce_threshold = 120
    config.evolution.reproduce_cost = 80
    config.evolution.mutation_std = 0.01

    # Training
    config.train.total_steps = TOTAL_STEPS
    config.train.num_envs = 32
    config.train.num_steps = 128
    config.log.wandb = False
    config.log.save_interval = 0

    # Adaptive gate ON with evolution-only control
    config.field.adaptive_gate = True
    config.field.evolutionary_gate_only = True
    config.field.gate_bias_mutation_std = 0.5
    config.field.channel_diffusion_rates = (0.15, 1.0, 0.0, 0.0)
    config.field.channel_decay_rates = (0.05, 1.0, 0.0, 0.0)

    # Patch marking for clean signal
    config.nest.patch_scaling_enabled = False
    config.nest.food_patch_marking = True

    return config


TASKS = [
    ("task_A_easy", build_task_a()),
    ("task_B_hard", build_task_b()),
]

print("=" * 60)
print("ADAPTIVE GATE DEMO")
print("=" * 60)
for name, cfg in TASKS:
    print(f"\n{name}:")
    print(f"  grid={cfg.env.grid_size}, food={cfg.env.num_food}")
    print(f"  hidden_food={cfg.env.hidden_food.enabled}")
    print(f"  adaptive_gate={cfg.field.adaptive_gate}")
    print(f"  evolutionary_gate_only={cfg.field.evolutionary_gate_only}")
    print(f"  food_obs={cfg.env.food_obs_enabled}")

ADAPTIVE GATE DEMO

task_A_easy:
  grid=14, food=12
  hidden_food=False
  adaptive_gate=True
  evolutionary_gate_only=True
  food_obs=True

task_B_hard:
  grid=14, food=6
  hidden_food=True
  adaptive_gate=True
  evolutionary_gate_only=True
  food_obs=True


In [3]:
import gc
import time
import numpy as np
from src.training.parallel_train import ParallelTrainer

all_results = {}

for task_name, config in TASKS:
    print(f"\n{'='*60}")
    print(f"TRAINING: {task_name}")
    print(f"{'='*60}")

    checkpoint_dir = f"{CHECKPOINT_BASE}/{task_name}"
    seed_ids = list(range(NUM_SEEDS))

    steps_per_iter = config.train.num_envs * config.train.num_steps * config.evolution.max_agents
    num_iterations = max(1, TOTAL_STEPS // steps_per_iter)

    print(f"Steps/iter: {steps_per_iter:,}")
    print(f"Iterations: {num_iterations}")

    try:
        t0 = time.time()
        trainer = ParallelTrainer(
            config=config,
            num_seeds=NUM_SEEDS,
            seed_ids=seed_ids,
            checkpoint_dir=checkpoint_dir,
            master_seed=42,
        )

        metrics = trainer.train(
            num_iterations=num_iterations,
            checkpoint_interval_minutes=30,
            resume=False,
            print_interval=5,
        )

        elapsed = time.time() - t0

        all_results[task_name] = {
            'metrics': metrics,
            'time': elapsed,
            'success': True,
        }

        print(f"\n{task_name} completed in {elapsed/60:.1f} minutes")

    except Exception as e:
        print(f"FAILED: {e}")
        import traceback
        traceback.print_exc()
        all_results[task_name] = {'success': False, 'error': str(e)}

    finally:
        try:
            del trainer
        except Exception:
            pass
        gc.collect()
        try:
            if hasattr(jax, 'clear_caches'):
                jax.clear_caches()
        except Exception:
            pass

print("\n" + "="*60)
print("ALL TASKS COMPLETE")
print("="*60)


TRAINING: task_A_easy
Steps/iter: 131,072
Iterations: 22
Parallel Multi-Seed Training
Seeds: [0, 1, 2]
Checkpoint dir: /content/drive/MyDrive/emergence-lab/adaptive_gate_demo_v2/task_A_easy
Checkpoint interval: 30 minutes

Initializing fresh state...
  Initializing seed 0...
  Initializing seed 1...
  Initializing seed 2...
JIT compiling parallel train step...
JIT compilation done (72.5s)

Steps per iteration: 131072
Starting from step: 0

Iter     0/22 | Step     131072 | Reward: 0.6950 +/- 0.0021 | Loss: 226.0858 | 0.25 it/s
Iter     5/22 | Step     786432 | Reward: 1.0226 +/- 0.0980 | Loss: 36.4511 | 0.25 it/s
Iter    10/22 | Step    1441792 | Reward: 1.3125 +/- 0.1072 | Loss: 38.0718 | 0.25 it/s
Iter    15/22 | Step    2097152 | Reward: 1.1278 +/- 0.2923 | Loss: 32.2378 | 0.25 it/s
Iter    20/22 | Step    2752512 | Reward: 0.8923 +/- 0.1180 | Loss: 25.5923 | 0.25 it/s
Iter    21/22 | Step    2883584 | Reward: 1.1195 +/- 0.2874 | Loss: 31.8661 | 0.25 it/s

Saving final checkpoints 

In [4]:
import numpy as np

print("=" * 60)
print("ADAPTIVE GATE RESULTS")
print("=" * 60)

gate_values = {}

for name, result in all_results.items():
    if not result.get('success'):
        print(f"\n{name}: FAILED — {result.get('error', 'unknown')}")
        continue

    m = result['metrics']
    rewards = np.array(m.get('mean_reward', [0.0]), dtype=float)
    population = np.array(m.get('population_size', [0.0]), dtype=float)
    pickups = np.array(m.get('num_pickups', [0.0]), dtype=float)
    deliveries = np.array(m.get('num_deliveries', [0.0]), dtype=float)
    mean_gate = np.array(m.get('mean_gate', [0.0]), dtype=float)

    gate_values[name] = mean_gate

    print(f"\n--- {name} ---")
    print(f"  Reward:      {np.mean(rewards):.4f} +/- {np.std(rewards):.4f}")
    print(f"  Population:  {np.mean(population):.2f}")
    print(f"  Deliveries:  {np.mean(deliveries):.1f}")
    print(f"  Mean Gate:   {np.mean(mean_gate):.4f} +/- {np.std(mean_gate):.4f}")
    print(f"  Time:        {result.get('time', 0)/60:.1f} min")

# === THE VERDICT ===
if 'task_A_easy' in gate_values and 'task_B_hard' in gate_values:
    gate_a = np.mean(gate_values['task_A_easy'])
    gate_b = np.mean(gate_values['task_B_hard'])
    ratio = gate_b / gate_a if gate_a > 0.001 else float('inf')

    print("\n" + "=" * 60)
    print("ADAPTIVE GATE COMPARISON")
    print("=" * 60)
    print(f"  Task A (easy) mean gate:  {gate_a:.4f}")
    print(f"  Task B (hard) mean gate:  {gate_b:.4f}")
    print(f"  Ratio (B/A):              {ratio:.2f}x")
    print()

    if ratio >= 3.0 and gate_b > 0.1:
        print("VERDICT: ADAPTIVE ENGINE WORKS")
        print(f"Gate opens {ratio:.1f}x more in coordination task.")
        print("The system activates stigmergy only when economically justified.")
    elif gate_b > gate_a and gate_b > 0.05:
        print("VERDICT: PARTIAL ADAPTATION")
        print(f"Gate is higher in Task B ({gate_b:.4f} vs {gate_a:.4f})")
        print("but ratio < 3x. May need longer training or stronger incentive.")
    elif gate_a < 0.1 and gate_b < 0.1:
        print("VERDICT: GATE STAYS CLOSED IN BOTH TASKS")
        print("PPO suppresses the field regardless of task difficulty.")
        print("The field mechanism needs architectural changes.")
    else:
        print(f"VERDICT: INCONCLUSIVE (A={gate_a:.4f}, B={gate_b:.4f})")
    print("=" * 60)

ADAPTIVE GATE RESULTS

--- task_A_easy ---
  Reward:      1.1195 +/- 0.2874
  Population:  31.98
  Deliveries:  1441.7
  Mean Gate:   0.4999 +/- 0.0000
  Time:        2.9 min

--- task_B_hard ---
  Reward:      1.0441 +/- 0.2733
  Population:  32.00
  Deliveries:  1249.7
  Mean Gate:   0.4999 +/- 0.0006
  Time:        3.1 min

ADAPTIVE GATE COMPARISON
  Task A (easy) mean gate:  0.4999
  Task B (hard) mean gate:  0.4999
  Ratio (B/A):              1.00x

VERDICT: PARTIAL ADAPTATION
Gate is higher in Task B (0.4999 vs 0.4999)
but ratio < 3x. May need longer training or stronger incentive.
